In [ ]:
# instando bibliotcas
#!py -m pip install pandas
#!py -m pip install sqlalchemy psycopg2-binary
#!py -m pip install pyodbc
#!py -m pip install sqlalchemy
#!py -m pip install python-dotenv

In [7]:
import pandas as pd

# O prefixo 'r'  é usado para garantir que as barras invertidas sejam tratadas 
# literalmente, evitando erros com sequências de escape do Python.
caminho_do_arquivo = r'C:\Users\Rodrigo\Desktop\MBA UNIESP\PROJETO DW\projeto_posgraduacao_dw\dados\brutos\world_imdb_movies_top_movies_per_year.csv' 

# Lendo o arquivo csv 
try:
    df = pd.read_csv(caminho_do_arquivo, sep=',', encoding='utf-8')
    
    # Exibe as 5 primeiras linhas
    print("Arquivo lido com sucesso! Aqui estão as primeiras linhas:")
    display(df.head())

#arquivo não encontrado
except FileNotFoundError:
    print("ERRO: O arquivo não foi encontrado. Verifique o caminho.")

#verificando outros erros
except Exception as e:
    print(f"OUTROS ERROS ENCONTRADOS.\n Detalhe: {e}")

Arquivo lido com sucesso! Aqui estão as primeiras linhas:


,id,title,link,year,duration,rating_mpa,rating_imdb,vote,budget,gross_world_wide,...,writer,star,genre,country_origin,filming_location,production_company,language,win,nomination,oscar
0,tt0073470,Strip Nude for Your Killer,https://www.imdb.com/title/tt0073470,1975,1h 38m,Not Rated,5.6,33000.0,NaN,NaN,...,"Andrea Bianchi, Massimo Felisatti","Edwige Fenech, Nino Castelnuovo, Femi Benussi","Erotic Thriller, Giallo, Slasher Horror, Horro...",Italy,"Milan, Lombardia, Italy",FRAL Cinematografica,Italian,0,0,0
1,tt0072764,Carry on Behind,https://www.imdb.com/title/tt0072764,1975,1h 30m,Not Rated,5.6,31000.0,NaN,NaN,...,Dave Freeman,"Elke Sommer, Kenneth Williams, Bernard Bresslaw","Parody, Slapstick, Comedy",United Kingdom,"Bad Godesberg Way, Maidenhead, Berkshire, Engl...","The Rank Organisation, Peter Rogers Productions",English,0,0,0
2,tt0131526,"So Young, So Lovely, So Vicious...",https://www.imdb.com/title/tt0131526,1975,1h 30m,NaN,4.9,487.0,NaN,NaN,...,"Silvio Amadio, Roberto Natale","Gloria Guida, Dagmar Lassander, Fred Robsahm","Drama, Mystery, Thriller",Italy,"Sardinia, Italy",Domiziana Internazionale Cinematografica,Italian,0,0,0
3,tt0073335,Mahogany,https://www.imdb.com/title/tt0073335,1975,1h 49m,PG,6.1,23000.0,NaN,NaN,...,"John Byrum, Toni Amber","Diana Ross, Billy Dee Williams, Anthony Perkins","Drama, Romance",United States,Marshall Field Co Department Store 111 N Sta...,"Motown Productions, Nikor Productions, Paramou...","English, Italian",0,0,1
4,tt0073006,Foreplay,https://www.imdb.com/title/tt0073006,1975,1h 15m,R,4.0,185.0,NaN,NaN,...,"Dan Greenburg, Jack Richardson, Bruce Jay Frie...","Irwin Corey, Pat Paulsen, Deborah Loomis","Parody, Satire, Sketch Comedy, Comedy",United States,NaN,SynFrank Enterprises,English,0,0,0


*CONEXAO POSTGRES -- O PADRÃO SERA SQL SERVER*

In [ ]:
from sqlalchemy import create_engine, text 

# VARIÁVEIS DE CONEXÃO
DB_HOST = 'localhost'       
DB_NAME = 'prod_filmes' 
DB_USER = 'postgres'
DB_PASS = '112358'
DB_PORT = '5432'            

# VARIÁVEIS DE INSERÇÃO
nome_da_tabela = 'imdb_filmes'
if_exists_option = 'replace' 
DB_SCHEMA = 'raw'           

# CRIAR A STRING DE CONEXÃO
conexao_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# CRIAR O MOTOR (ENGINE) DA CONEXÃO
try:
    engine = create_engine(conexao_string)
    print("Conexão com o banco de dados estabelecida.")
    
    # GARANTIR A EXISTÊNCIA DO SCHEMA
    with engine.connect() as connection:
        create_schema_sql = text(f"CREATE SCHEMA IF NOT EXISTS {DB_SCHEMA};")
        connection.execute(create_schema_sql)
        connection.commit()
    
    print(f"Schema '{DB_SCHEMA}' verificado e criado.")

    # INSERIR OS DADOS
    df.to_sql(
        name=nome_da_tabela,
        con=engine,
        if_exists=if_exists_option,
        index=False,
        chunksize=1000,
        schema=DB_SCHEMA 
    )
    
    # mensagem de sucesso
    nome_completo = f'{DB_SCHEMA}.{nome_da_tabela}'
    print(f"\nSucesso! {len(df)} linhas inseridas na tabela '{nome_completo}' no PostgreSQL com a opção '{if_exists_option}'.")

except NameError:
    # Capturar erro se o DataFrame 'df' não estiver definido
    print("\nERRO: O DataFrame 'df' não está definido ou carregado.")
except Exception as e:
    print("\nERRO: Não foi possível conectar ou inserir os dados.")
    print(f"Detalhe do erro: {e}")
    print("Verifique:")
    print("1. Se o PostgreSQL está rodando.")
    print("2. As suas credenciais (usuário, senha, nome do banco).")
    print("3. Se o nome do banco de dados (DB_NAME) realmente existe.")

finally:
    # 6. FECHAR A CONEXÃO
    if 'engine' in locals():
        engine.dispose()

CONEXAO SQL SERVER

In [ ]:
from sqlalchemy import create_engine, text

# quote_plus é Necessário para codificar a senha, se houver caracteres especiais
from urllib.parse import quote_plus 

#variaveis do banco
DB_HOST = r'DESKTOP-ERJHOB6\SQLEXPRESS' 
DB_NAME = 'prod_filmes'                 
DB_USER = 'sa'                          
DB_PASS = '112358'              
DRIVER = 'ODBC Driver 17 for SQL Server' 
DB_SCHEMA = 'raw'

# variáveis de inserção
nome_da_tabela = 'imdb_filmes'
if_exists_option = 'replace' 


# Codificar a senha para evitar erros quanddo trasmitida
senha_codificada = quote_plus(DB_PASS)

# criar a string de conexão para sql server 
conexao_string = (
    f"mssql+pyodbc://{DB_USER}:{senha_codificada}@{DB_HOST}/{DB_NAME}?"
    f"driver={DRIVER}"
)

# Cria o objeto que define como se conectar ao banco de dados,
# gerenciando o dialeto (SQL Server) e o pool de conexões.
try:
    engine = create_engine(conexao_string)
    print("Motor de conexão (Engine) criado com sucesso.")
    
    # Criar o schema se não existir
    with engine.connect() as connection:
        create_schema_sql = text(f"IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = '{DB_SCHEMA}') EXEC('CREATE SCHEMA {DB_SCHEMA}');")
        connection.execute(create_schema_sql)
        connection.commit()
    
    print(f"Schema '{DB_SCHEMA}' - Verificado e criando caso não exista")

    # Carrega o DataFrame no SQL Server, substituindo a tabela existente 
    # e otimizando a transferência em lotes de 3000 linhas.
    df.to_sql(
        name=nome_da_tabela,
        con=engine,
        if_exists=if_exists_option, # aqui irá substituir o que já existe na base ((replace acima))
        index=False,
        chunksize=3000,
        schema=DB_SCHEMA 
    )
    
    # criando o nome da tabela com o schema
    nome_schema_e_tabela = f'{DB_SCHEMA}.{nome_da_tabela}'

    #validando a quantidade de linhas
    try:
        linhas_inseridas = len(df)
    except NameError:
        print("AVISO: Não foi possível obter a contagem de linhas.")
        linhas_inseridas = None
        
    print(f"\n{linhas_inseridas} linhas inseridas na tabela '{nome_schema_e_tabela}' no SQL Server \nRealizado um '{if_exists_option}'nos dados existentes.")

except NameError:
    print("\nERRO: O DataFrame 'df' não está definido ou carregado. \n Certifique-se de que o DataFrame existe antes de chamar df.to_sql().")
except Exception as e:
    print("\nERRO: Não foi possível conectar ou inserir os dados no SQL Server.")
    print(f"Detalhe do erro: {e}")
    print("Verifique:")
    print("1. A sua **SENHA** na variável DB_PASS.")
    print("2. A instalação do **pyodbc** e do **ODBC Driver 17 for SQL Server**.")
    print("3. Se o SQL Server está rodando e aceitando conexões.")

finally:
    # FECHAR A CONEXÃO
    if 'engine' in locals():
        engine.dispose()
        print("Conexão do motor (engine) fechada.")

Motor de conexão (Engine) criado com sucesso.
Schema 'raw' - Verificado e criando caso não exista

Sucesso! 33600 linhas inseridas na tabela 'raw.imdb_filmes' no SQL Server realizado um 'replace'nos dados existentes.
Conexão do motor (engine) fechada.
